In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test, save, load, get_loss_history
from Models.basic_models import average_model, two_layer_basic, post_process

set_figsize(20, 15)

Training directory found, 36 series
Validation directory found, 6 series
Testing directory found, 10 series


In [2]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# "Pixl", "Slice", or "None"
preproc_type = "pixel"

validation = TAVR_3_Frame("__valid", preproc=preproc_type, preload=False)
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc=preproc_type, preload=False)
train_loader = tavr_dataloader(training,batch_size=6, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_layer_basic()
post_proc = post_process(kind=preproc_type).to(device=device)
loss_fn = batch_l2_loss()

# CHANGE TO NAME OF JUPYTER NOTEBOOK
model_name = "Model 5 (Basic+PixelNorm) R1"

using device: cuda


In [3]:
learning_rate = 3e-3
momentum = 0.90
reg = 1e-7

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=reg, nesterov=True)

In [4]:
LOAD = True
iteration_num = 420

if LOAD:
    load(model_name, iteration_num, model, optimizer)
    loss_history = get_loss_history(model_name)
    model.to(device=device)
    # I don't know why these lines are necessary
    # or even what the hell they do
    # but they are
    if str(device) == 'cuda':
        for state in optimizer.state.values():
            for k, v in state.items():
                state[k] = v.cuda()
else:
    loss_history = None

model loaded from model_checkpoints/Model 5 (Basic+PixelNorm) R1/Model 5 (Basic+PixelNorm) R1-420


In [ ]:
# If multiple GPU
# model = nn.DataParallel(model)

In [5]:
train(model, post_proc, optimizer, train_loader, val_loader, loss_fn, device, 
         model_name, loss_history,
          epochs=2, print_every=30, print_level=4, lr_decay=1)


Iteration 420, loss = 0.1161, corrected loss = 54.0283
Validation loss 50.5887 over 81 frames
model saved to model_checkpoints/Model 5 (Basic+PixelNorm) R1/Model 5 (Basic+PixelNorm) R1-420
conv_a1.weight,   	norm: 4.1521e+00, 	update norm: 7.6968e-04 	Update/norm: 1.8537e-04
conv_a1.bias,   	norm: 3.9000e-01, 	update norm: 1.4662e-04 	Update/norm: 3.7593e-04
conv_b1.weight,   	norm: 4.0642e+00, 	update norm: 7.7102e-04 	Update/norm: 1.8971e-04
conv_b1.bias,   	norm: 3.2725e-01, 	update norm: 1.1400e-04 	Update/norm: 3.4837e-04
final.weight,   	norm: 7.7837e-01, 	update norm: 4.8138e-04 	Update/norm: 6.1845e-04
final.bias,   	norm: 7.1204e-02, 	update norm: 4.1649e-06 	Update/norm: 5.8492e-05

... 0.1066... 0.1053... 0.1176
Iter 430... 0.1093... 0.1077... 0.1166
Iter 440... 0.1196... 0.1163... 0.1226
Iteration 450, loss = 0.1027, corrected loss = 47.7695
Validation loss 50.0477 over 81 frames
model saved to model_checkpoints/Model 5 (Basic+PixelNorm) R1/Model 5 (Basic+PixelNorm) R1-450

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, post_proc, val_loader, loss_fn, device)
test(ave_model, post_proc, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [val_seq[0][0]]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [ave_frame[0][0],  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)